# Dirty Dataset
This example illustrate how to perform meta-blocking on a dirty dataset (data deduplication), so when we have one dataset that contains duplicates.

In [130]:
import sparker
import random

In [131]:
import pandas as pd

In [132]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

## Load the data
sparkER provides wrappers to load CSV and JSON files.

*real_id_field* is the field that contains the identifier of the record.

In [133]:
# Profiles contained in the first dataset
profiles = sparker.CSVWrapper.load_profiles('file:///home/aditi/BigData/AAI_Project/Entity_Resolution/Entity_Resolution_Project/sparker_try/sample1_sample2.csv',
                                            start_id_from=0,
                                            separator=",", header=True,
                                            real_id_field = "id",
                                            source_id=0)


In [134]:
# Profiles contained in the first dataset
# profiles = sparker.JSONWrapper.load_profiles('/Final_Project/sample/Df2.json', real_id_field = "id")

Let's visualize a profile to check if they are correctly loaded

In [135]:
print(profiles.take(1)[0])

{'profile_id': 0, 'attributes': [{'key': 'name', 'value': 'Omkar   jori'}, {'key': 'fathername', 'value': 'Nathu jori'}, {'key': 'mothername', 'value': 'Aruna jori'}, {'key': 'dateofbirth', 'value': '1999-01-13'}, {'key': 'location', 'value': 'kothrud'}, {'key': 'city', 'value': 'Pune'}, {'key': 'state', 'value': 'Maharastra'}, {'key': 'country', 'value': 'India'}, {'key': 'pincode', 'value': '411038'}, {'key': 'gender', 'value': 'M'}, {'key': 'age', 'value': '22'}, {'key': 'qualification', 'value': 'BE in mechnical'}, {'key': 'hobby', 'value': 'swimming'}], 'original_id': '1', 'source_id': 0}


In [136]:
profiles.take(5)
profiles.toDF().show(1364)

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{name, Omkar   j...|          1|         0|        0|
|[{name, Mr Gupta}...|          2|         1|        0|
|[{name, Amruta  A...|          3|         2|        0|
|[{name, varun jha...|          4|         3|        0|
|[{name, Aditi Pat...|          5|         4|        0|
|[{name, Bhumik Pa...|          6|         5|        0|
|[{name, Miss Amru...|          7|         6|        0|
|[{name, Omkar  na...|          8|         7|        0|
|[{name, Rohit gup...|          9|         8|        0|
|[{name, Argade Am...|         10|         9|        0|
|[{name, varun jha...|         11|        10|        0|
|[{name, Patil Adi...|         12|        11|        0|
|[{name, Bhumika H...|         13|        12|        0|
|[{name, Mrs Deshm...|         14|        13|        0|
|[{name, Amruta Ar...|         15|        14|   

Extract the max id (it will be used later)

In [137]:
# Max profile id
max_profile_id = profiles.map(lambda profile: profile.profile_id).max()

In [138]:
max_profile_id

22


### Groundtruth (optional)
If you have a groundtruth you can measure the performance of each step.

When you load the groundtruth it contains the original profiles IDs, it is necessary to convert it to use the IDs assigned to each profile by Spark.

In [139]:
# Loads the groundtruth, takes as input the path of the file and the names of the attributes that represent
# respectively the id of profiles of the first dataset and the id of profiles of the second dataset
gt = sparker.CSVWrapper.load_groundtruth('file:///home/aditi/BigData/AAI_Project/Entity_Resolution/Entity_Resolution_Project/sparker_try/sample_groundtruth.csv', 'id1', 'id2')

In [140]:
# # Converts the groundtruth by replacing original IDs with those given by Spark
new_gt = sparker.Converters.convert_groundtruth(gt, profiles)

In [141]:
# We can explore some pairs
random.sample(new_gt, 2)

/tmp/ipykernel_9300/3362377442.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(new_gt, 2)


[(0, 7), (16, 17)]

## Blocking
Now we can perform blocking.

By default sparkER performs token blocking, but it is possible to provide a different blocking function.

In the following example each token is splitted in ngrams of size 4 that are used for blocking.

In [142]:
blocks = sparker.Blocking.create_blocks(profiles, 
                                        blocking_method=sparker.BlockingKeysStrategies.ngrams_blocking,
                                        ngram_size=4)
print("Number of blocks",blocks.count())

Number of blocks 608


Let's continue by using the standard token blocking

In [143]:
# blocks = sparker.Blocking.create_blocks(profiles)
# print("Number of blocks",blocks.count())

## Block cleaning

sparkER implements two block cleaning strategies:

* Block purging: discard the largest blocks that involve too many comparisons, the parameter must be >= 1. A lower value mean a more aggressive purging.
* Block cleaning: removes for every profile the largest blocks in which it appears. The parameter is in range ]0, 1\[. A lower value mean a more aggressive cleaning.

In [144]:
# Perfoms the purging
blocks_purged = sparker.BlockPurging.block_purging(blocks, 1.025)

In [145]:
# Performs the cleaning
(profile_blocks, profile_blocks_filtered, blocks_after_filtering) = sparker.BlockFiltering.block_filtering_quick(blocks_purged, 0.8)

If you have the groundtruth, after every blocking step it is possible to check which are the performance of the blocking collection.

In [146]:
recall, precision, cmp_n = sparker.Utils.get_statistics(blocks_after_filtering, max_profile_id, new_gt)

print("Recall", recall)
print("Precision", precision)
print("Number of comparisons", cmp_n)

Recall 1.0
Precision 0.044
Number of comparisons 250


## Meta-blocking
Meta-blocking can be used to further refine the block collection removing superfluous comparisons.

SparkER implements different kind of meta-blocking algorithms, you can find the descriptions in our paper.


For every partition of the RDD the pruning algorithm returns as output a triplet that contains:

* The number of edges
* The number of matches (only if the groundtruth is provided)
* The retained edges

To perform the meta-blocking first some data structures have to be created.

In [147]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [148]:
block_index_map = blocks_after_filtering.map(lambda b : (b.block_id, b.profiles)).collectAsMap()
block_index = sc.broadcast(block_index_map)

# This is only needed for certain weight measures
profile_blocks_size_index = sc.broadcast(profile_blocks_filtered.map(lambda pb : (pb.profile_id, len(pb.blocks))).collectAsMap())

# Broadcasted groundtruth
gt_broadcast = sc.broadcast(new_gt)

### Weighted Node Pruning

In [149]:
WNP_results = sparker.WNP.wnp(
    profile_blocks_filtered,
    block_index,
    max_profile_id,
    #separator_ids,
    weight_type=sparker.WeightTypes.CBS,
    groundtruth=gt_broadcast,
    profile_blocks_size_index=profile_blocks_size_index,
    comparison_type=sparker.ComparisonTypes.OR
)

num_edges = WNP_results.map(lambda x: x[0]).sum()
num_matches = WNP_results.map(lambda x: x[1]).sum()
recall = num_matches / len(new_gt)
precision = (num_matches / num_edges)
print("Recall", recall)
print("Precision", precision)
print("Number of comparisons", num_edges)
wnp_f1 = 2 * ((precision * recall) / (precision + recall))
print("F1 score", wnp_f1)

Recall 1.0
Precision 0.15942028985507245
Number of comparisons 69
F1 score 0.27499999999999997


### Reciprocal Weighted Node Pruning

In [150]:
RWNP_results = sparker.WNP.wnp(
    profile_blocks_filtered,
    block_index,
    max_profile_id,
    #separator_ids,
    weight_type=sparker.WeightTypes.CBS,
    groundtruth=gt_broadcast,
    profile_blocks_size_index=profile_blocks_size_index,
    comparison_type=sparker.ComparisonTypes.AND
)
num_edges = RWNP_results.map(lambda x: x[0]).sum()
num_matches = RWNP_results.map(lambda x: x[1]).sum()

recall = num_matches / len(new_gt)
precision = (num_matches / num_edges)
print("Recall", recall)
print("Precision", precision)
print("Number of comparisons", num_edges)
rwnp_f1 = 2 * ((precision * recall) / (precision + recall))
print("F1 score", rwnp_f1)

Recall 1.0
Precision 0.28205128205128205
Number of comparisons 39
F1 score 0.44000000000000006


### Weighted Edge Pruning

In [151]:
WEP_results = sparker.WEP.wep(
    profile_blocks_filtered,
    block_index,
    max_profile_id,
    #separator_ids,
    weight_type=sparker.WeightTypes.CBS,
    groundtruth=gt_broadcast,
    profile_blocks_size_index=profile_blocks_size_index
)
num_edges = WEP_results.map(lambda x: x[0]).sum()
num_matches = WEP_results.map(lambda x: x[1]).sum()

recall = num_matches / len(new_gt)
precision = (num_matches / num_edges)
print("Recall", recall)
print("Precision", precision)
print("Number of comparisons", num_edges)
wep_f1 = 2 * ((precision * recall) / (precision + recall))
print("F1 score", wep_f1)

Recall 1.0
Precision 0.18333333333333332
Number of comparisons 60
F1 score 0.30985915492957744


### Cardinality Edge Pruning

In [152]:
CEP_results = sparker.CEP.cep(
    profile_blocks_filtered,
    block_index,
    max_profile_id,
    #separator_ids,
    weight_type=sparker.WeightTypes.CBS,
    groundtruth=gt_broadcast,
    profile_blocks_size_index=profile_blocks_size_index
)
num_edges = CEP_results.map(lambda x: x[0]).sum()
num_matches = CEP_results.map(lambda x: x[1]).sum()

recall = num_matches / len(new_gt)
precision = (num_matches / num_edges)
print("Recall", recall)
print("Precision", precision)
print("Number of comparisons", num_edges)
cep_f1 = 2 * ((precision * recall) / (precision + recall))
print("F1 score", cep_f1)

Recall 1.0
Precision 0.044
Number of comparisons 250
F1 score 0.08429118773946359


In [153]:
f1_score_list = [wnp_f1, rwnp_f1, wep_f1, cep_f1]
f1_score_final = max(f1_score_list)

print("\n\n\n F1_score_final", f1_score_final)

if (f1_score_final == wnp_f1):
    final_score_result = WNP_results
elif (f1_score_final == rwnp_f1):
    final_score_result = RWNP_results
elif (f1_score_final == wep_f1):
    final_score_result = WEP_results
elif (f1_score_final == cep_f1):
    final_score_result = CEP_results




 F1_score_final 0.44000000000000006


## Collecting edges after meta-blocking
As mentioned before, the third element of the tuples returned by the meta-blocking contains the edges.


Edges are weighted according to the weight strategy provided to the meta-blocking.

In [154]:
edges = final_score_result.flatMap(lambda x: x[2])

edges.take(10)

[(0, 7, 91),
 (0, 2, 30),
 (0, 4, 30),
 (0, 11, 30),
 (7, 9, 34),
 (7, 11, 29),
 (1, 8, 71),
 (2, 9, 108),
 (2, 13, 101),
 (2, 6, 62)]

In [155]:
edges_df = pd.DataFrame(edges.collect())

In [156]:
edges_df

0   1    2
0    0   2   30
1    0   4   30
2    0   7   91
3    0  11   30
4    7   9   34
5    7  11   29
6    1   8   71
7    2   9  108
8    2  13  101
9    2   6   62
10   2   7   33
11   5  11   36
12   5  12   57
13   5  16   31
14   8  17   23
15   8  10   20
16   9  13  120
17  14  15   79
18  15  16   24
19  16  17   62
20  16  20   24
21   4   7   29
22   4  11   87
23   4  12   27
24   4  16   36
25   4  20   26
26   4   5   41
27  11  12   27
28  11  16   35
29  11  20   26
30  12  22   23
31  12  15   23
32  12  16   27
33  20  22   17
34   3  10  108
35   3   8   18
36   6   9   62
37   6  13   57
38  18  19   43

In [157]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
final_score_result.toDF(newColumns).show()

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|          4|            1|[{0, 7, 91}, {0, ...|
|          2|            0|[{7, 9, 34}, {7, ...|
|          1|            1|        [{1, 8, 71}]|
|          4|            3|[{2, 9, 108}, {2,...|
|          3|            1|[{5, 12, 57}, {5,...|
|          2|            0|[{8, 10, 20}, {8,...|
|          1|            0|      [{9, 13, 120}]|
|          0|            0|                  []|
|          1|            1|      [{14, 15, 79}]|
|          1|            0|      [{15, 16, 24}]|
|          2|            1|[{16, 17, 62}, {1...|
|          0|            0|                  []|
|          6|            1|[{4, 5, 41}, {4, ...|
|          3|            0|[{11, 12, 27}, {1...|
|          3|            0|[{12, 15, 23}, {1...|
|          1|            0|      [{20, 22, 17}]|
|          2|            1|[{3, 10, 108}, {3...|
|          2|       

In [158]:
max_by_group = (edges
  .map(lambda x: (x[0], x))  
   .reduceByKey(lambda x1, x2: max(x1, x2, key=lambda x: x[-1]))
   .values())

In [159]:
max_by_group.take(10)

[(0, 7, 91),
 (7, 9, 34),
 (1, 8, 71),
 (2, 9, 108),
 (5, 12, 57),
 (8, 17, 23),
 (9, 13, 120),
 (14, 15, 79),
 (15, 16, 24),
 (16, 17, 62)]

In [160]:
dataCollect = max_by_group.collect()
df =  pd.DataFrame(dataCollect)
df

0   1    2
0    0   7   91
1    7   9   34
2    1   8   71
3    2   9  108
4    5  12   57
5    8  17   23
6    9  13  120
7   14  15   79
8   15  16   24
9   16  17   62
10   4  11   87
11  11  16   35
12  12  16   27
13  20  22   17
14   3  10  108
15   6   9   62
16  18  19   43

In [161]:
df.rename(columns = {0:'P_id', 1:'N_id',
                              2:'weight'}, inplace = True)
df.head()

P_id  N_id  weight
0     0     7      91
1     7     9      34
2     1     8      71
3     2     9     108
4     5    12      57

In [162]:
n = df.filter(["P_id","N_id","weight"])
n = pd.DataFrame(n)
n

P_id  N_id  weight
0      0     7      91
1      7     9      34
2      1     8      71
3      2     9     108
4      5    12      57
5      8    17      23
6      9    13     120
7     14    15      79
8     15    16      24
9     16    17      62
10     4    11      87
11    11    16      35
12    12    16      27
13    20    22      17
14     3    10     108
15     6     9      62
16    18    19      43

In [163]:
# avg_val = n["weight"].mean()
# avg_val = round(avg_val, 0)
# avg_val

# new_df = n[(n['weight'] > avg_val)]

In [164]:
# new_df = n[(n['weight']>=1)]
# new_df["P_id"] = new_df["P_id"] + 1
# new_df["N_id"] = new_df["N_id"] + 1
# new_df

In [165]:
max1 = n["weight"].max()
max1 = round(max1,0)
max1

120

In [166]:
max_percentage = (max1*30)/100
new_df = n[(n['weight']>max_percentage)]
new_df["P_id"] = new_df["P_id"] + 1
new_df["N_id"] = new_df["N_id"] + 1
new_df

/tmp/ipykernel_9300/3154471759.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["P_id"] = new_df["P_id"] + 1
/tmp/ipykernel_9300/3154471759.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["N_id"] = new_df["N_id"] + 1


P_id  N_id  weight
0      1     8      91
2      2     9      71
3      3    10     108
4      6    13      57
6     10    14     120
7     15    16      79
9     17    18      62
10     5    12      87
14     4    11     108
15     7    10      62
16    19    20      43

In [167]:
new_df["c_id"] = [x for x in range(0, len(new_df))]
new_df

col = ["P_id", "N_id", "weight", "cluster_id"]
df_weight = spark.createDataFrame(new_df, schema=col)
df_weight.show(20)

+----+----+------+----------+
|P_id|N_id|weight|cluster_id|
+----+----+------+----------+
|   1|   8|    91|         0|
|   2|   9|    71|         1|
|   3|  10|   108|         2|
|   6|  13|    57|         3|
|  10|  14|   120|         4|
|  15|  16|    79|         5|
|  17|  18|    62|         6|
|   5|  12|    87|         7|
|   4|  11|   108|         8|
|   7|  10|    62|         9|
|  19|  20|    43|        10|
+----+----+------+----------+



/tmp/ipykernel_9300/4287424093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["c_id"] = [x for x in range(0, len(new_df))]


In [168]:
new_df = new_df.sort_values(by=['P_id'])

In [169]:
new_df.head(20)

P_id  N_id  weight  c_id
0      1     8      91     0
2      2     9      71     1
3      3    10     108     2
14     4    11     108     8
10     5    12      87     7
4      6    13      57     3
15     7    10      62     9
6     10    14     120     4
7     15    16      79     5
9     17    18      62     6
16    19    20      43    10

In [171]:
col = ["P_id","N_id","weight","c_id"]
df_weight = spark.createDataFrame(new_df, schema=col)
df_weight.show()

+----+----+------+----+
|P_id|N_id|weight|c_id|
+----+----+------+----+
|   1|   8|    91|   0|
|   2|   9|    71|   1|
|   3|  10|   108|   2|
|   4|  11|   108|   8|
|   5|  12|    87|   7|
|   6|  13|    57|   3|
|   7|  10|    62|   9|
|  10|  14|   120|   4|
|  15|  16|    79|   5|
|  17|  18|    62|   6|
|  19|  20|    43|  10|
+----+----+------+----+



In [172]:
from pyspark.sql import Window
import pyspark.sql.functions as f

In [173]:
df_weight = df_weight.withColumn('cluster_id', f.first('c_id').over(Window.partitionBy('N_id')))
df_weight.show()

+----+----+------+----+----------+
|P_id|N_id|weight|c_id|cluster_id|
+----+----+------+----+----------+
|   1|   8|    91|   0|         0|
|   2|   9|    71|   1|         1|
|   3|  10|   108|   2|         2|
|   7|  10|    62|   9|         2|
|   4|  11|   108|   8|         8|
|   5|  12|    87|   7|         7|
|   6|  13|    57|   3|         3|
|  10|  14|   120|   4|         4|
|  15|  16|    79|   5|         5|
|  17|  18|    62|   6|         6|
|  19|  20|    43|  10|        10|
+----+----+------+----+----------+



In [174]:
df_weight_pd = df_weight.toPandas()
df_weight_pd

P_id  N_id  weight  c_id  cluster_id
0      1     8      91     0           0
1      2     9      71     1           1
2      3    10     108     2           2
3      7    10      62     9           2
4      4    11     108     8           8
5      5    12      87     7           7
6      6    13      57     3           3
7     10    14     120     4           4
8     15    16      79     5           5
9     17    18      62     6           6
10    19    20      43    10          10

In [175]:
pid_df = pd.DataFrame(df_weight_pd.filter(["P_id","N_id","weight","cluster_id"]))
pid_df

P_id  N_id  weight  cluster_id
0      1     8      91           0
1      2     9      71           1
2      3    10     108           2
3      7    10      62           2
4      4    11     108           8
5      5    12      87           7
6      6    13      57           3
7     10    14     120           4
8     15    16      79           5
9     17    18      62           6
10    19    20      43          10

In [176]:
nid_df = pd.DataFrame(new_df.iloc[:,1])
nid_df = nid_df.reset_index()
nid_df = nid_df.drop(["index"], axis = 1)
nid_df

N_id
0      8
1      9
2     10
3     11
4     12
5     13
6     10
7     14
8     16
9     18
10    20

In [177]:
# nid_dict = {}
# for i in range(len(nid_df)):
#     nid_dict[i] = nid_df[i]

In [178]:
df3 = pd.merge(pid_df, nid_df, how='outer', left_on='P_id', right_on='N_id')
df3 = df3.drop_duplicates()
df3 = df3.dropna(thresh=3)
df3 = df3.fillna(0)
df3 = df3.reset_index()
df3 = df3.drop(["index"], axis = 1)
df3

P_id  N_id_x  weight  cluster_id  N_id_y
0    1.0     8.0    91.0         0.0     0.0
1    2.0     9.0    71.0         1.0     0.0
2    3.0    10.0   108.0         2.0     0.0
3    7.0    10.0    62.0         2.0     0.0
4    4.0    11.0   108.0         8.0     0.0
5    5.0    12.0    87.0         7.0     0.0
6    6.0    13.0    57.0         3.0     0.0
7   10.0    14.0   120.0         4.0    10.0
8   15.0    16.0    79.0         5.0     0.0
9   17.0    18.0    62.0         6.0     0.0
10  19.0    20.0    43.0        10.0     0.0

In [179]:
lst = [x for x in df3["N_id_x"]]

for i in range(len(df3)):
    a = (df3.loc[i,"P_id"])
    if(a in lst):
        j = lst.index(a)
        df3.loc[i,"cluster_id"] = df3.loc[j,"cluster_id"]
        #print(df3.loc[i,"P_id"],df3.loc[i,"cluster_id"])
        
df_weight = df3.drop(["N_id_y"], axis = 1)
df_weight

P_id  N_id_x  weight  cluster_id
0    1.0     8.0    91.0         0.0
1    2.0     9.0    71.0         1.0
2    3.0    10.0   108.0         2.0
3    7.0    10.0    62.0         2.0
4    4.0    11.0   108.0         8.0
5    5.0    12.0    87.0         7.0
6    6.0    13.0    57.0         3.0
7   10.0    14.0   120.0         2.0
8   15.0    16.0    79.0         5.0
9   17.0    18.0    62.0         6.0
10  19.0    20.0    43.0        10.0

In [180]:
col = ["P_id","N_id","weight","cluster_id"]
df_weight = spark.createDataFrame(df_weight, schema=col)
df_weight.show()

+----+----+------+----------+
|P_id|N_id|weight|cluster_id|
+----+----+------+----------+
| 1.0| 8.0|  91.0|       0.0|
| 2.0| 9.0|  71.0|       1.0|
| 3.0|10.0| 108.0|       2.0|
| 7.0|10.0|  62.0|       2.0|
| 4.0|11.0| 108.0|       8.0|
| 5.0|12.0|  87.0|       7.0|
| 6.0|13.0|  57.0|       3.0|
|10.0|14.0| 120.0|       2.0|
|15.0|16.0|  79.0|       5.0|
|17.0|18.0|  62.0|       6.0|
|19.0|20.0|  43.0|      10.0|
+----+----+------+----------+



In [181]:
ss = SparkSession(sc)


In [182]:
df1 = ss.read.option("header",True)\
        .option("inferSchema",
                True)\
        .csv('file:///home/aditi/BigData/AAI_Project/Entity_Resolution/Entity_Resolution_Project/sparker_try/sample1_sample2.csv')
df1.show(5)

+---+--------------+--------------+-------------+-----------+--------------------+------+-------------+-------+-------+------+---+---------------+--------+
| id|          name|    fathername|   mothername|dateofbirth|            location|  city|        state|country|pincode|gender|age|  qualification|   hobby|
+---+--------------+--------------+-------------+-----------+--------------------+------+-------------+-------+-------+------+---+---------------+--------+
|  1|  Omkar   jori|    Nathu jori|   Aruna jori| 1999-01-13|             kothrud|  Pune|   Maharastra|  India| 411038|     M| 22|BE in mechnical|swimming|
|  2|      Mr Gupta|       Suresh | Neelam gupta| 1994-03-11|                null|  Orai|Uttar pradesh|  India| 285001|     M| 27| diploma in CSE|  eating|
|  3|Amruta  Argade|Diwakar Argade|Sunita Argade| 1993-09-14| sankul, Near zea...|  Pune|   Maharastra|  India| 411041|     F| 28|           ENTC| dancing|
|  4|    varun jhaa|    Arvind Jha|  Urvashi Jha| 1998-07-15| M-

In [183]:
from pyspark.sql.functions import monotonically_increasing_id


In [184]:
common_df = df1.join(df_weight,df1.id == df_weight.P_id,"inner")

In [185]:
pd_common_df = common_df.toPandas()
pd_common_df

id                   name       fathername     mothername dateofbirth  \
0    1           Omkar   jori       Nathu jori     Aruna jori  1999-01-13   
1    2               Mr Gupta          Suresh    Neelam gupta  1994-03-11   
2    3         Amruta  Argade   Diwakar Argade  Sunita Argade  1993-09-14   
3    7            Miss Amruta             None        Sunita         None   
4    4             varun jhaa       Arvind Jha    Urvashi Jha  1998-07-15   
5    5            Aditi Patil     Hemant Patil   Yogita bonde  1999-08-22   
6    6           Bhumik Patil    Patil Hemaant   Lalita bonde  2003-09-06   
7   10  Argade Amruta Diwakar   Diwakar Argade  Sunita Argade  1993-09-14   
8   15          Amruta Argade      Anad Argade  Shweta Argade  2003-08-16   
9   17         Amruta Bhosale  Pradeep Bhosale   Lata Bhosale  1999-06-19   
10  19           Aditee Yadav   Shrikant Yadav    Mamta Yadav        None   

                                             location               city  \
0                                             kothrud               Pune   
1                                                None               Orai   
2               sankul, Near zeal engineering college               Pune   
3               sankul, Near zeal engineering college  Panchavati Pashan   
4                                 M-04,Vaishali Nagar             Jaipur   
5                                       khanda colony             Panvel   
6                                         Sai darshan          NewPanvel   
7   Keshav rukmini sankul, Near zeal engineering c...               Pune   
8                                       Anand Society            Solapur   
9                                   Sai Krupa housing             Panvel   
10                                   Gulmohar Society              Panji   

            state country   pincode gender   age    qualification     hobby  \
0      Maharastra   India  411038.0      M  22.0  BE in mechnical  swimming   
1   Uttar pradesh   India  285001.0      M  27.0   diploma in CSE    eating   
2      Maharastra   India  411041.0      F  28.0             ENTC   dancing   
3            None   India       NaN      F   NaN             ENTC   dancing   
4      Rajasthan    India  302012.0      M  24.0        BE in CSE  sleeping   
5      Maharastra   India  410206.0      F  22.0       BE in EXTC    eating   
6     Maharashtra   India  400026.0      M  18.0   Btech EXTC NIT  irritate   
7      Maharastra   India  411041.0      F  30.0       BE in ENTC   dancing   
8     Maharashtra   India  420036.0      F  18.0              CSE   Cooking   
9    Maharashtra    India  410206.0      F  23.0            BE IT   Cycling   
10            Goa   India  656565.0      F  28.0         Btech CS    Gaming   

    P_id  N_id  weight  cluster_id  
0    1.0   8.0    91.0         0.0  
1    2.0   9.0    71.0         1.0  
2    3.0  10.0   108.0         2.0  
3    7.0  10.0    62.0         2.0  
4    4.0  11.0   108.0         8.0  
5    5.0  12.0    87.0         7.0  
6    6.0  13.0    57.0         3.0  
7   10.0  14.0   120.0         2.0  
8   15.0  16.0    79.0         5.0  
9   17.0  18.0    62.0         6.0  
10  19.0  20.0    43.0        10.0

In [186]:
common_N = df1.join(df_weight, df1.id == df_weight.N_id, "inner")

In [187]:
pd_common_N = common_N.toPandas()
pd_common_N

id                   name            fathername     mothername  \
0    8      Omkar  nathu jori            Nathu jori     Aruna jori   
1    9            Rohit gupta  Suresh chandra gupta   Neelam gupta   
2   10  Argade Amruta Diwakar        Diwakar Argade  Sunita Argade   
3   10  Argade Amruta Diwakar        Diwakar Argade  Sunita Argade   
4   11              varun jha            Arvind Jha    Urvashi Jha   
5   12            Patil Aditi          Hemant Patil   Yogita Patil   
6   13   Bhumika Hemant Patil          Hemant Patil           None   
7   14           Mrs Deshmukh        Diwakar Argade  Sunita Argade   
8   16    Amruta Anand Argade           Anad Argade  Shweta Argade   
9   18     Ms Amruta Lokhande       Pradeep Bhosale   Lata Bhosale   
10  20         Aditi Deshmukh        Shrikant Yadav    Mamta Yadav   

   dateofbirth                                           location  \
0   1999-01-13                        Kalashree MD Rivera Bhugaon   
1   1994-03-11                                        tulsi nagar   
2   1993-09-14  Keshav rukmini sankul, Near zeal engineering c...   
3   1993-09-14  Keshav rukmini sankul, Near zeal engineering c...   
4   1998-07-15               M-04,Ashiana Mangalam,Vaishali Nagar   
5   1999-08-22                                     Gurudev Arcade   
6   2000-09-16                                        Sai darshan   
7   1993-09-14  Keshav rukmini sankul, Near zeal engineering c...   
8   2003-08-16                                       Gandhi nagar   
9   1999-06-19                           Sector 10, Neelam Arcade   
10        None                                      Bakers Street   

             city          state country   pincode gender  age  \
0            Pune     Maharastra   India  411041.0      M   22   
1            Orai  Uttar pradesh   India  285001.0      M   27   
2            Pune     Maharastra   India  411041.0      F   30   
3            Pune     Maharastra   India  411041.0      F   30   
4          Jaipur     Rajasthan    India  302012.0      M   24   
5          Panvel     Maharastra   India  410206.0      F   22   
6     Navi Mumbai    Maharashtra   India  400046.0      F   21   
7           Baner     Maharastra   India  411041.0      F   28   
8         Solapur    Maharashtra   India  420036.0      F   18   
9       Hyderabad     Telangana    India  512007.0      F   23   
10  New York city       New York      US       NaN      F   28   

       qualification             hobby  P_id  N_id  weight  cluster_id  
0    BE in mechnical          swimming   1.0   8.0    91.0         0.0  
1          BE in ECE            runing   2.0   9.0    71.0         1.0  
2         BE in ENTC           dancing   3.0  10.0   108.0         2.0  
3         BE in ENTC           dancing   7.0  10.0    62.0         2.0  
4          BE in CSE          sleeping   4.0  11.0   108.0         8.0  
5         BE in EXTC            eating   5.0  12.0    87.0         7.0  
6              Btech          sleeping   6.0  13.0    57.0         3.0  
7         BE in ENTC           dancing  10.0  14.0   120.0         2.0  
8   Computer science           Cooking  15.0  16.0    79.0         5.0  
9              BE IT  Cycling Teaching  17.0  18.0    62.0         6.0  
10          Btech CS        Game Dance  19.0  20.0    43.0        10.0

In [188]:
import functools


def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)


# unionAll
result3 = unionAll([common_df, common_N])
result3.show()

final = result3.toPandas()

final = final.drop(["P_id", "N_id"], axis=1)
final.head()

final_cluster = final.groupby(['cluster_id', 'id'])
final_cluster.first()

+---+--------------------+--------------------+-------------+-----------+--------------------+-----------------+-------------+-------+-------+------+----+----------------+--------+----+----+------+----------+
| id|                name|          fathername|   mothername|dateofbirth|            location|             city|        state|country|pincode|gender| age|   qualification|   hobby|P_id|N_id|weight|cluster_id|
+---+--------------------+--------------------+-------------+-----------+--------------------+-----------------+-------------+-------+-------+------+----+----------------+--------+----+----+------+----------+
|  1|        Omkar   jori|          Nathu jori|   Aruna jori| 1999-01-13|             kothrud|             Pune|   Maharastra|  India| 411038|     M|  22| BE in mechnical|swimming| 1.0| 8.0|  91.0|       0.0|
|  2|            Mr Gupta|             Suresh | Neelam gupta| 1994-03-11|                null|             Orai|Uttar pradesh|  India| 285001|     M|  27|  diploma 

name            fathername     mothername  \
cluster_id id                                                               
0.0        1            Omkar   jori            Nathu jori     Aruna jori   
           8       Omkar  nathu jori            Nathu jori     Aruna jori   
1.0        2                Mr Gupta               Suresh    Neelam gupta   
           9             Rohit gupta  Suresh chandra gupta   Neelam gupta   
2.0        3          Amruta  Argade        Diwakar Argade  Sunita Argade   
           7             Miss Amruta                  None        Sunita    
           10  Argade Amruta Diwakar        Diwakar Argade  Sunita Argade   
           14           Mrs Deshmukh        Diwakar Argade  Sunita Argade   
3.0        6            Bhumik Patil         Patil Hemaant   Lalita bonde   
           13   Bhumika Hemant Patil          Hemant Patil           None   
5.0        15          Amruta Argade           Anad Argade  Shweta Argade   
           16    Amruta Anand Argade           Anad Argade  Shweta Argade   
6.0        17         Amruta Bhosale       Pradeep Bhosale   Lata Bhosale   
           18     Ms Amruta Lokhande       Pradeep Bhosale   Lata Bhosale   
7.0        5             Aditi Patil          Hemant Patil   Yogita bonde   
           12            Patil Aditi          Hemant Patil   Yogita Patil   
8.0        4              varun jhaa            Arvind Jha    Urvashi Jha   
           11              varun jha            Arvind Jha    Urvashi Jha   
10.0       19           Aditee Yadav        Shrikant Yadav    Mamta Yadav   
           20         Aditi Deshmukh        Shrikant Yadav    Mamta Yadav   

              dateofbirth                                           location  \
cluster_id id                                                                  
0.0        1   1999-01-13                                            kothrud   
           8   1999-01-13                        Kalashree MD Rivera Bhugaon   
1.0        2   1994-03-11                                               None   
           9   1994-03-11                                        tulsi nagar   
2.0        3   1993-09-14              sankul, Near zeal engineering college   
           7         None              sankul, Near zeal engineering college   
           10  1993-09-14  Keshav rukmini sankul, Near zeal engineering c...   
           14  1993-09-14  Keshav rukmini sankul, Near zeal engineering c...   
3.0        6   2003-09-06                                        Sai darshan   
           13  2000-09-16                                        Sai darshan   
5.0        15  2003-08-16                                      Anand Society   
           16  2003-08-16                                       Gandhi nagar   
6.0        17  1999-06-19                                  Sai Krupa housing   
           18  1999-06-19                           Sector 10, Neelam Arcade   
7.0        5   1999-08-22                                      khanda colony   
           12  1999-08-22                                     Gurudev Arcade   
8.0        4   1998-07-15                                M-04,Vaishali Nagar   
           11  1998-07-15               M-04,Ashiana Mangalam,Vaishali Nagar   
10.0       19        None                                   Gulmohar Society   
           20        None                                      Bakers Street   

                            city          state country   pincode gender  \
cluster_id id                                                              
0.0        1                Pune     Maharastra   India  411038.0      M   
           8                Pune     Maharastra   India  411041.0      M   
1.0        2                Orai  Uttar pradesh   India  285001.0      M   
           9                Orai  Uttar pradesh   India  285001.0      M   
2.0        3                Pune     Maharastra   India  411041.0      F   
           7   Panchavati Pashan           N

In [201]:
df = pd.concat(map(lambda x: x[1], final_cluster))

df = df.reset_index().drop(["index","id"],axis =1)

new_col = [x for x in range(0,len(df))]
df.insert(loc=0, column='id', value=new_col)

df1 = spark.createDataFrame(df)


df1.write.option('header',True).csv("hdfs://localhost:9000/ER_Project/stage/output",mode = 'overwrite')

spark.read.csv("hdfs://localhost:9000/ER_Project/stage/output", header=True).show()
print("File written successfully on HDFS .")

In [202]:
df

id                   name            fathername     mothername  \
0    0           Omkar   jori            Nathu jori     Aruna jori   
1    1      Omkar  nathu jori            Nathu jori     Aruna jori   
2    2               Mr Gupta               Suresh    Neelam gupta   
3    3            Rohit gupta  Suresh chandra gupta   Neelam gupta   
4    4         Amruta  Argade        Diwakar Argade  Sunita Argade   
5    5            Miss Amruta                  None        Sunita    
6    6  Argade Amruta Diwakar        Diwakar Argade  Sunita Argade   
7    7  Argade Amruta Diwakar        Diwakar Argade  Sunita Argade   
8    8  Argade Amruta Diwakar        Diwakar Argade  Sunita Argade   
9    9           Mrs Deshmukh        Diwakar Argade  Sunita Argade   
10  10           Bhumik Patil         Patil Hemaant   Lalita bonde   
11  11   Bhumika Hemant Patil          Hemant Patil           None   
12  12          Amruta Argade           Anad Argade  Shweta Argade   
13  13    Amruta Anand Argade           Anad Argade  Shweta Argade   
14  14         Amruta Bhosale       Pradeep Bhosale   Lata Bhosale   
15  15     Ms Amruta Lokhande       Pradeep Bhosale   Lata Bhosale   
16  16            Aditi Patil          Hemant Patil   Yogita bonde   
17  17            Patil Aditi          Hemant Patil   Yogita Patil   
18  18             varun jhaa            Arvind Jha    Urvashi Jha   
19  19              varun jha            Arvind Jha    Urvashi Jha   
20  20           Aditee Yadav        Shrikant Yadav    Mamta Yadav   
21  21         Aditi Deshmukh        Shrikant Yadav    Mamta Yadav   

   dateofbirth                                           location  \
0   1999-01-13                                            kothrud   
1   1999-01-13                        Kalashree MD Rivera Bhugaon   
2   1994-03-11                                               None   
3   1994-03-11                                        tulsi nagar   
4   1993-09-14              sankul, Near zeal engineering college   
5         None              sankul, Near zeal engineering college   
6   1993-09-14  Keshav rukmini sankul, Near zeal engineering c...   
7   1993-09-14  Keshav rukmini sankul, Near zeal engineering c...   
8   1993-09-14  Keshav rukmini sankul, Near zeal engineering c...   
9   1993-09-14  Keshav rukmini sankul, Near zeal engineering c...   
10  2003-09-06                                        Sai darshan   
11  2000-09-16                                        Sai darshan   
12  2003-08-16                                      Anand Society   
13  2003-08-16                                       Gandhi nagar   
14  1999-06-19                                  Sai Krupa housing   
15  1999-06-19                           Sector 10, Neelam Arcade   
16  1999-08-22                                      khanda colony   
17  1999-08-22                                     Gurudev Arcade   
18  1998-07-15                                M-04,Vaishali Nagar   
19  1998-07-15               M-04,Ashiana Mangalam,Vaishali Nagar   
20        None                                   Gulmohar Society   
21        None                                      Bakers Street   

                 city          state country   pincode gender   age  \
0                Pune     Maharastra   India  411038.0      M  22.0   
1                Pune     Maharastra   India  411041.0      M  22.0   
2                Orai  Uttar pradesh   India  285001.0      M  27.0   
3                Orai  Uttar pradesh   India  285001.0      M  27.0   
4                Pune     Maharastra   India  411041.0      F  28.0   
5   Panchavati Pashan           None   India       NaN      F   NaN   
6                Pune     Maharastra   India  411041.0      F  30.0   
7                Pune     Maharastra   India  411041.0      F  30.0   
8                Pune     Maharastra   India  411041.0      F  30.0   
9               Baner     Maharastra   India  411041.0      F  28.0   
10          NewPanvel   

In [203]:
df1.show(5)

+---+-----------------+--------------------+-------------+-----------+--------------------+----+-------------+-------+--------+------+----+---------------+--------+------+----------+
| id|             name|          fathername|   mothername|dateofbirth|            location|city|        state|country| pincode|gender| age|  qualification|   hobby|weight|cluster_id|
+---+-----------------+--------------------+-------------+-----------+--------------------+----+-------------+-------+--------+------+----+---------------+--------+------+----------+
|  0|     Omkar   jori|          Nathu jori|   Aruna jori| 1999-01-13|             kothrud|Pune|   Maharastra|  India|411038.0|     M|22.0|BE in mechnical|swimming|  91.0|       0.0|
|  1|Omkar  nathu jori|          Nathu jori|   Aruna jori| 1999-01-13|Kalashree MD Rive...|Pune|   Maharastra|  India|411041.0|     M|22.0|BE in mechnical|swimming|  91.0|       0.0|
|  2|         Mr Gupta|             Suresh | Neelam gupta| 1994-03-11|               